In [1]:
!pip install pandas
!pip install scikit-learn
!pip install pycaret
!pip install mlflow
!pip install boto3
!pip install apache-airflow
!airflow db init
!airflow scheduler
!airflow webserver

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [2]:
# Import libraries needed,
import IPython
import os
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
from statistics import mode
from pycaret.regression import *
import mlflow
from mlflow.tracking import MlflowClient
from sklearn.model_selection import train_test_split
from sklearn import metrics

# download the dataset
# Directory of the raw data files
_data_root = './data/Diabetes'
# Path to the raw training data
_data_filepath = os.path.join(_data_root, 'Diabetes.csv')
# Download data
os.makedirs(_data_root, exist_ok=True)
if not os.path.isfile(_data_filepath):
    # https://archive.ics.uci.edu/ml/machine-learning-databases/covtype/
    url = 'https://docs.google.com/uc?export=download&confirm={{VALUE}}&id=1k5-1caezQ3zWJbKaiMULTGq-3sz6uThC'
    r = requests.get(url, allow_redirects=True, stream=True)
    open(_data_filepath, 'wb').write(r.content)



2.3.2


In [3]:
dataoriginal = pd.read_csv('data/Diabetes/Diabetes.csv')

In [6]:
datacopy = dataoriginal.copy() 
Rep = datacopy.replace('?', np.NaN) 
nacheck = Rep.isnull().sum() 
nacheck

encounter_id                    0
patient_nbr                     0
race                         2273
gender                          0
age                             0
weight                      98569
admission_type_id               0
discharge_disposition_id        0
admission_source_id             0
time_in_hospital                0
payer_code                  40256
medical_specialty           49949
num_lab_procedures              0
num_procedures                  0
num_medications                 0
number_outpatient               0
number_emergency                0
number_inpatient                0
diag_1                         21
diag_2                        358
diag_3                       1423
number_diagnoses                0
max_glu_serum                   0
A1Cresult                       0
metformin                       0
repaglinide                     0
nateglinide                     0
chlorpropamide                  0
glimepiride                     0
acetohexamide 

In [7]:
datacopy= datacopy.drop(['weight','payer_code','medical_specialty'],axis=1)

In [9]:
datacopy['30readmit'] = np.where(datacopy['readmitted'] == 'NO', 0, 1)

In [10]:
datacopy.groupby('30readmit').size()

30readmit
0    54864
1    46902
dtype: int64

In [11]:
datacopy = datacopy[((datacopy.discharge_disposition_id != 11) & 
                                          (datacopy.discharge_disposition_id != 13) &
                                          (datacopy.discharge_disposition_id != 14) & 
                                          (datacopy.discharge_disposition_id != 19) & 
                                          (datacopy.discharge_disposition_id != 20) & 
                                          (datacopy.discharge_disposition_id != 21))] 

In [13]:
# Cleaning the data, replacing the null values in numeric data by 0 and object data by unknown,

numcolumn = datacopy.select_dtypes(include = [np.number]).columns
objcolumn = datacopy.select_dtypes(include = ['object']).columns

In [14]:
# Substituting 0 and unknown,

datacopy[numcolumn] = datacopy[numcolumn].fillna(0)
datacopy[objcolumn] = datacopy[objcolumn].fillna("unknown")

In [16]:
#Encoding the data,

def map_now():
    listname = [('infections', 139),
                ('neoplasms', (239 - 139)),
                ('endocrine', (279 - 239)),
                ('blood', (289 - 279)),
                ('mental', (319 - 289)),
                ('nervous', (359 - 319)),
                ('sense', (389 - 359)),
                ('circulatory', (459-389)),
                ('respiratory', (519-459)),
                ('digestive', (579 - 519)),
                ('genitourinary', (629 - 579)),
                ('pregnancy', (679 - 629)),
                ('skin', (709 - 679)),
                ('musculoskeletal', (739 - 709)),
                ('congenital', (759 - 739)),
                ('perinatal', (779 - 759)),
                ('ill-defined', (799 - 779)),
                ('injury', (999 - 799))]
    
    
    dictcout = {}
    count = 1
    for name, num in listname:
        for i in range(num):
            dictcout.update({str(count): name})  
            count += 1
    return dictcout
  

def codemap(df, codes):
    import pandas as pd
    namecol = df.columns.tolist()
    for col in namecol:
        temp = [] 
        for num in df[col]:           
            if ((num is None) | (num in ['unknown', '?']) | (pd.isnull(num))): temp.append('unknown')
            elif(num.upper()[0] == 'V'): temp.append('supplemental')
            elif(num.upper()[0] == 'E'): temp.append('injury')
            else: 
                lkup = num.split('.')[0]
                temp.append(codes[lkup])           
        df.loc[:, col] = temp               
    return df 


listcol = ['diag_1', 'diag_2', 'diag_3']
codes = map_now()
datacopy[listcol] = codemap(datacopy[listcol], codes)

In [18]:
data1 = datacopy.drop(['encounter_id', "patient_nbr", 'admission_type_id','readmitted'], axis =1) 

In [20]:
#Normalization of the data,

listnormal = ['time_in_hospital', 'num_lab_procedures', 'num_procedures', 'num_medications',
                     'number_outpatient', 'number_emergency', 'number_inpatient', 'number_diagnoses']

from sklearn.preprocessing import StandardScaler

normal = StandardScaler()

data1[listnormal] = normal.fit_transform(data1[listnormal])

,discharge_disposition_id,admission_source_id,time_in_hospital,num_lab_procedures,num_procedures,num_medications,number_outpatient,number_emergency,number_inpatient,number_diagnoses,30readmit
count,99343.000000,99343.000000,9.934300e+04,9.934300e+04,9.934300e+04,9.934300e+04,9.934300e+04,9.934300e+04,9.934300e+04,9.934300e+04,99343.000000
mean,3.517882,5.731083,6.894932e-17,-8.632969e-17,-4.756358e-17,1.773800e-17,-7.438515e-18,-5.650411e-17,-2.660700e-17,1.773800e-16,0.471256
std,5.184359,4.065029,1.000005e+00,1.000005e+00,1.000005e+00,1.000005e+00,1.000005e+00,1.000005e+00,1.000005e+00,1.000005e+00,0.499176
min,1.000000,1.000000,-1.138438e+00,-2.137026e+00,-7.835647e-01,-1.850439e+00,-2.918627e-01,-2.116216e-01,-5.005746e-01,-3.298145e+00,0.000000
25%,1.000000,1.000000,-8.015554e-01,-6.071886e-01,-7.835647e-01,-7.386238e-01,-2.918627e-01,-2.116216e-01,-5.005746e-01,-7.221572e-01,0.000000
50%,1.000000,7.000000,-1.277904e-01,5.574070e-02,-1.962887e-01,-1.209485e-01,-2.918627e-01,-2.116216e-01,-5.005746e-01,3.082380e-01,0.000000
75%,3.000000,7.000000,5.459746e-01,7.186700e-01,3.909872e-01,4.967267e-01,-2.918627e-01,-2.116216e-01,2.928105e-01,8.234355e-01,1.000000
max,28.000000,25.000000,3.241035e+00,4.543262e+00,2.740091e+00,8.032365e+00,3.290615e+01,8.083523e+01,1.616051e+01,4.429819e+00,1.000000


In [21]:
#Let's store readmitted in y and rest of the columns in X,

Y = data1['30readmit']
X = data1.drop(['30readmit'], axis =1)
X = pd.get_dummies(X)

In [22]:


X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = .2, 
                                                random_state = 7, stratify = Y)

In [28]:
#Splitting the data into training and vallidation data sets. The training data will contain 80 % of the data and validation will contain remaining 20%
print("shape of Xtrain,Xtest:",X_train.shape,X_test.shape)

# Check data type
print(X_train.dtypes)

# Check missing values
print(X_train.isnull().sum())


shape of Xtrain,Xtest: (79474, 169) (19869, 169)
discharge_disposition_id           int64
admission_source_id                int64
time_in_hospital                 float64
num_lab_procedures               float64
num_procedures                   float64
                                  ...   
metformin-pioglitazone_Steady      uint8
change_Ch                          uint8
change_No                          uint8
diabetesMed_No                     uint8
diabetesMed_Yes                    uint8
Length: 169, dtype: object
discharge_disposition_id         0
admission_source_id              0
time_in_hospital                 0
num_lab_procedures               0
num_procedures                   0
                                ..
metformin-pioglitazone_Steady    0
change_Ch                        0
change_No                        0
diabetesMed_No                   0
diabetesMed_Yes                  0
Length: 169, dtype: int64


In [24]:
print("Missing values in X_train:", X_train.isnull().sum().sum())
print("Missing values in Y_train:", Y_train.isnull().sum())


Missing values in X_train: 0
Missing values in Y_train: 0


In [25]:
os.environ['TZ'] = 'America/Bogota'
os.environ['MLFLOW_S3_ENDPOINT_URL'] = "http://10.43.102.111:9000"
os.environ['AWS_ACCESS_KEY_ID'] = 'admin'
os.environ['AWS_SECRET_ACCESS_KEY'] = 'supersecret'

In [33]:
# Connect to MLflow
mlflow.set_tracking_uri("http://10.43.102.111:5000")

# Enable autologging in MLflow
mlflow.autolog(log_model_signatures=True, log_input_examples=True)

# Set up the data
s = setup(data1, target = '30readmit', transform_target = True, log_experiment = True, experiment_name = 'Experimento proyecto final MLOPS 2023')
# Compare models
best_models = compare_models(n_select=10, sort='R2', fold=5)

# comparar modelos
best = compare_models()
# finalize the model
final_best = finalize_model(best)
# save model to disk
save_model(final_best, 'final_best_production_Last_Project_MLOPS')

# Registra el modelo en MLflow
mlflow.sklearn.log_model(final_best, 'final_best', registered_model_name='final_best_production_Last_Project_MLOPS')


# Add the best model to production
model_name = "final_best_production_Last_Project_MLOPS"

# Create an MLflow client
client = MlflowClient()

# Get information about all versions without stage, from the most recent to the oldest
all_versions = client.get_latest_versions(model_name, stages=None)
print('Versions:', all_versions)

# Check if there are registered versions of the model
if len(all_versions) > 0:
    # Get the latest registered version of the model
    latest_version = all_versions[-1].version
    print(f"The latest version of the model is version {latest_version}.")
else:
    print("There are no registered versions of the model.")

# Transition the model to production
client.transition_model_version_stage(
    name=model_name,
    version=latest_version,
    stage="Production"
)


2023/05/23 22:32:52 INFO mlflow.tracking.fluent: Autologging successfully enabled for sklearn.
2023/05/23 22:32:52 INFO mlflow.tracking.fluent: Autologging successfully enabled for lightgbm.
2023/05/23 22:32:52 WARNING mlflow.utils.autologging_utils: You are using an unsupported version of statsmodels. If you encounter errors during autologging, try upgrading / downgrading statsmodels to a supported version, or try upgrading MLflow.
2023/05/23 22:32:52 INFO mlflow.tracking.fluent: Autologging successfully enabled for statsmodels.


,Description,Value
0,Session id,7692
1,Target,30readmit
2,Target type,Regression
3,Original data shape,"(99343, 44)"
4,Transformed data shape,"(99343, 161)"
5,Transformed train set shape,"(69540, 161)"
6,Transformed test set shape,"(29803, 161)"
7,Ordinal features,9
8,Numeric features,10
9,Categorical features,33


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
lightgbm,Light Gradient Boosting Machine,0.4419,0.2361,0.4859,0.0521,0.3283,0.5921,1.1440
gbr,Gradient Boosting Regressor,0.4472,0.2386,0.4884,0.0422,0.3297,0.5998,6.7620
rf,Random Forest Regressor,0.4430,0.2407,0.4906,0.0336,0.3325,0.5887,32.2420
et,Extra Trees Regressor,0.4428,0.2446,0.4946,0.0179,0.3361,0.5847,51.9140
ada,AdaBoost Regressor,0.4578,0.2456,0.4956,0.0138,0.3351,0.6109,0.9480
ridge,Ridge Regression,0.4566,0.2497,0.4996,-0.0023,0.3342,0.6152,0.8040
br,Bayesian Ridge,0.4572,0.2497,0.4997,-0.0026,0.3342,0.6162,1.6980
omp,Orthogonal Matching Pursuit,0.4598,0.2516,0.5016,-0.0103,0.3353,0.6199,0.4580
huber,Huber Regressor,0.4553,0.2634,0.5132,-0.0575,0.3349,0.6152,3.4320
lasso,Lasso Regression,0.4906,0.2652,0.5150,-0.0648,0.3474,0.6571,0.5560


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
lightgbm,Light Gradient Boosting Machine,0.4419,0.2360,0.4858,0.0526,0.3282,0.5921,1.2710
gbr,Gradient Boosting Regressor,0.4472,0.2385,0.4884,0.0424,0.3297,0.5998,8.2160
rf,Random Forest Regressor,0.4430,0.2407,0.4906,0.0336,0.3325,0.5886,38.2070
et,Extra Trees Regressor,0.4427,0.2445,0.4945,0.0183,0.3360,0.5852,59.6050
ada,AdaBoost Regressor,0.4558,0.2448,0.4947,0.0173,0.3346,0.6070,1.1760
lr,Linear Regression,0.4109,0.2255,0.4504,-0.0052,0.3008,0.5537,3.8620
ridge,Ridge Regression,0.4566,0.2505,0.5004,-0.0056,0.3342,0.6153,0.7130
br,Bayesian Ridge,0.4572,0.2505,0.5005,-0.0057,0.3342,0.6162,2.6770
omp,Orthogonal Matching Pursuit,0.4597,0.2522,0.5021,-0.0124,0.3353,0.6199,0.6680
lasso,Lasso Regression,0.4906,0.2652,0.5150,-0.0648,0.3474,0.6571,2.0010


Transformation Pipeline and Model Successfully Saved


Successfully registered model 'final_best_production_Last_Project_MLOPS'.
2023/05/23 23:10:59 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation. Model name: final_best_production_Last_Project_MLOPS, version 1
Created version '1' of model 'final_best_production_Last_Project_MLOPS'.


Versions: [<ModelVersion: aliases=[], creation_timestamp=1684901483026, current_stage='None', description='', last_updated_timestamp=1684901483026, name='final_best_production_Last_Project_MLOPS', run_id='cb63684a5c184b9b9f1cdc970ed5dcc0', run_link='', source='s3://mlop-s3/artifacts/9/cb63684a5c184b9b9f1cdc970ed5dcc0/artifacts/final_best', status='READY', status_message='', tags={}, user_id='', version='1'>]
The latest version of the model is version 1.


NameError: name 'best_model' is not defined

In [42]:
print(data1.columns)

Index(['race', 'gender', 'age', 'discharge_disposition_id',
       'admission_source_id', 'time_in_hospital', 'num_lab_procedures',
       'num_procedures', 'num_medications', 'number_outpatient',
       'number_emergency', 'number_inpatient', 'diag_1', 'diag_2', 'diag_3',
       'number_diagnoses', 'max_glu_serum', 'A1Cresult', 'metformin',
       'repaglinide', 'nateglinide', 'chlorpropamide', 'glimepiride',
       'acetohexamide', 'glipizide', 'glyburide', 'tolbutamide',
       'pioglitazone', 'rosiglitazone', 'acarbose', 'miglitol', 'troglitazone',
       'tolazamide', 'examide', 'citoglipton', 'insulin',
       'glyburide-metformin', 'glipizide-metformin',
       'glimepiride-pioglitazone', 'metformin-rosiglitazone',
       'metformin-pioglitazone', 'change', 'diabetesMed', '30readmit'],
      dtype='object')


In [60]:
# Get the feature names used during training
training_columns = s.data.columns[:-1]  # Exclude the target column

# Check the columns in your test dataset (X_test)
print(X_test.columns)

# Drop columns not present in the training data
columns_to_drop = set(X_test.columns) - set(training_columns)
X_test.drop(columns_to_drop, axis=1, inplace=True)

# Add missing columns with default values
columns_to_add = set(training_columns) - set(X_test.columns)
for column in columns_to_add:
    X_test[column] = 'Unknown'  # Assign a default value to the new column

# Ensure the order of columns matches the training data
X_test = X_test[training_columns]

# Predict using the aligned test dataset
y_pred = final_best.predict(X_test)



Index(['discharge_disposition_id', 'admission_source_id', 'time_in_hospital',
       'num_lab_procedures', 'num_procedures', 'num_medications',
       'number_outpatient', 'number_emergency', 'number_inpatient',
       'number_diagnoses'],
      dtype='object')


In [61]:
# Evaluate the best model on the test set
#print(X_test)
y_pred = final_best.predict(X_test)
metricR2 = metrics.r2_score(Y_test, y_pred)
print("R2 Score:", metricR2)
print(f"This model was trained with version {latest_version} and has an R2 score of {metricR2}")

R2 Score: 0.04865532233090164
This model was trained with version 1 and has an R2 score of 0.04865532233090164


In [63]:
os.environ['MLFLOW_S3_ENDPOINT_URL'] = "http://10.43.102.111:9000"
os.environ['AWS_ACCESS_KEY_ID'] = 'admin'
os.environ['AWS_SECRET_ACCESS_KEY'] = 'supersecret'


import pandas as pd

data = [
    ['Caucasian', 'Female', '[10-20]', 25, 1, 41, 0, 1, 0, 0, 0,
     276, 500, 200, 9, 'None', 'None', 'No', 'No', 'No', 'No', 'No', 'No', 'YES', 'No',
     'No', 'No', 'No', 'No', 'No', 'No', 'No', 'No', 'No', 'No', 'No', 'No', 'No', 'No',
     'No', 'Ch', 'YES',0]
]

columns = [
    'race', 'gender', 'age', 'discharge_disposition_id',
    'admission_source_id', 'num_lab_procedures', 'num_procedures',
    'num_medications', 'number_outpatient', 'number_emergency', 'number_inpatient', 'diag_1',
    'diag_2', 'diag_3', 'number_diagnoses', 'max_glu_serum', 'A1Cresult', 'metformin',
    'repaglinide', 'nateglinide', 'chlorpropamide', 'glimepiride', 'acetohexamide', 'glipizide',
    'glyburide', 'tolbutamide', 'pioglitazone', 'rosiglitazone', 'acarbose', 'miglitol',
    'troglitazone', 'tolazamide', 'examide', 'citoglipton', 'insulin', 'glyburide-metformin',
    'glipizide-metformin', 'glimepiride-pioglitazone', 'metformin-rosiglitazone',
    'metformin-pioglitazone', 'change', 'diabetesMed','time_in_hospital'
]

input_df = pd.DataFrame(data, columns=columns)

# Connect to MLflow
# Create an MLflow client
client = MlflowClient()

mlflow.set_tracking_uri("http://10.43.102.111:5000")

# Get information about all versions from newest to oldest
all_versions = client.get_latest_versions("final_best_production_Last_Project_MLOPS", stages=["Production"])
latest_model_name = all_versions[-1].name
latest_model_stage = all_versions[-1].current_stage
latest_version = all_versions[-1].version
model_name = "final_best_production_Last_Project_MLOPS"

model_production_uri = "models:/{model_name}/production".format(model_name=model_name)
model_loaded = mlflow.pyfunc.load_model(model_uri=model_production_uri)

# Replace '?' with NaN
input_df.replace('?', np.nan, inplace=True)
prediction = model_loaded.predict(input_df)

print(f"El modelo que se utilizó para la inferencia es {latest_model_name} que está en {latest_model_stage}. Su versión es {latest_version} y la predicción es: {int(prediction[0])}")




2023/05/23 23:30:47 WARNING mlflow.pyfunc: Detected one or more mismatches between the model's dependencies and the current Python environment:
 - mlflow (current: 2.3.2, required: mlflow==2.3)
To fix the mismatches, call `mlflow.pyfunc.get_model_dependencies(model_uri)` to fetch the model's environment and install dependencies using the resulting environment file.


El modelo que se utilizó para la inferencia es final_best_production_Last_Project_MLOPS que está en Production. Su versión es 1 y la predicción es: 0
